# Truncated Digest Collision Analysis

The VMC Digest uses a truncated SHA-512 digest as an identifier for Location, Allele, Haplotype, and Genotype objects. This notebook discusses the choice of SHA-512 and the truncation length.

## Conclusions

* SHA-512 is often as fast or faster than other hash functions on modern 64-bit platforms due to the use of 64-bit instructions.
* 24 bytes of key is *ample* for sequence variation. Arguably, we could choose much smaller.

## References

[1] http://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf  
[2] https://tools.ietf.org/html/rfc3548#section-4  
[3] http://stackoverflow.com/a/4014407/342839  
[4] http://stackoverflow.com/a/22029380/342839  
[5] http://preshing.com/20110504/hash-collision-probabilities/  
[6] https://en.wikipedia.org/wiki/Birthday_problem


In [1]:
import hashlib
import math
import timeit

from IPython.display import display, Markdown

from vmcdemo.utils import _format_time

## Digest Timing

In [2]:
def blob(l):
    """return binary blob of length l (POSIX only)"""
    return open("/dev/urandom", "rb").read(l)

def digest(alg, blob):
    md = hashlib.new(alg)
    md.update(blob)
    return md.digest()



def magic_run1(alg, blob):
    t = %timeit -o digest(alg, blob)
    return t

def magic_tfmt(t):
    """format TimeitResult for table"""
    return "{a} ± {s} ([{b}, {w}])".format(
        a = _format_time(t.average),
        s = _format_time(t.stdev),
        b = _format_time(t.best),
        w = _format_time(t.worst),
    )

In [3]:
blob_lengths = [100, 1000, 10000, 100000, 1000000]
blobs = [blob(l) for l in blob_lengths]

In [33]:
table_rows = []
table_rows += [["algorithm"] + list(map(str,blob_lengths))]
table_rows += [["-"] * len(table_rows[0])]
for alg in sorted(hashlib.algorithms_guaranteed):
    r = [alg]
    for i in range(len(blobs)):
        blob = blobs[i]
        t = timeit.timeit(stmt='digest(alg, blob)', setup='from __main__ import alg, blob, digest', number=1000)
        r += [_format_time(t)]
    table_rows += [r]
table = "\n".join(["|".join(map(str,row)) for row in table_rows])
display(Markdown(table))

algorithm|100|1000|10000|100000|1000000
-|-|-|-|-|-
md5|1.43 ms|2.96 ms|18.8 ms|143 ms|1.42 s
sha1|1.06 ms|1.89 ms|11.2 ms|103 ms|1.02 s
sha224|1.21 ms|3.14 ms|23.1 ms|220 ms|2.2 s
sha256|1.28 ms|3.15 ms|23.3 ms|223 ms|2.2 s
sha384|1.19 ms|2.59 ms|16.1 ms|171 ms|1.5 s
sha512|1.17 ms|2.67 ms|16.7 ms|149 ms|1.47 s

** Conclusion: sha512 is as fast or faster than all but sha-1 (which is disfavored for cryptographic use). **

---
## Collision Analysis
According to [3], the probability `P` of a collision using `b` bits with `m` messages (sequences) is:

$$P(b, m) = m^2 / 2^{b+1}$$

Note that the collision probability depends on the number of messages, but not their size.  Solving for the number of messages:

$$m(b, P) = \sqrt{P * 2^{b+1}}$$

Solving for the minimum number of *bits* `b` as a function of an expected number of sequences `m` and a tolerance for collisions of `P`:

$$b(m, P) = \log_2{\left(\frac{m^2}{P}\right)} - 1$$

In [38]:
def B(P, m):
    """return the number of *bytes* needed to achieve a collision probability
    P for m messages"""
    return math.ceil((math.log2(m / P) - 1) / 8 / 3) * 3

In [39]:
m_bins = [1E6, 1E9, 1E12, 1E15, 1E18, 1E21, 1E24]
P_bins = [1E-24, 1E-21, 1E-18, 1E-15, 1E-12, 1E-9, 0.5]

In [40]:
table_rows = []
table_rows += [["#m"] + ["P<={P}".format(P=P) for P in P_bins]]
table_rows += [["-"] * len(table_rows[0])]
for n_m in m_bins:
    table_rows += [["{:g}".format(n_m)] + [B(P, n_m) for P in P_bins]]
table = "\n".join(["|".join(map(str,row)) for row in table_rows])
display(Markdown(table))

#m|P<=1e-24|P<=1e-21|P<=1e-18|P<=1e-15|P<=1e-12|P<=1e-09|P<=0.5
-|-|-|-|-|-|-|-
1e+06|15|12|12|9|9|9|3
1e+09|15|15|12|12|9|9|6
1e+12|15|15|15|12|12|9|6
1e+15|18|15|15|15|12|12|9
1e+18|18|18|15|15|15|12|9
1e+21|21|18|18|15|15|15|9
1e+24|21|21|18|18|15|15|12

### Discussion

Space considerations: The digest will be used for many objects and consume significant space at scale.
